In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32,[None, 784])
X_img = tf.reshape(X,[-1,28,28,1])
Y = tf.placeholder(tf.float32,[None, 10])

In [3]:
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1], padding='SAME')
print L1
L1 = tf.nn.relu(L1)
print L1
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print L1

Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [4]:
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1], padding='SAME')
print L2
L2 = tf.nn.relu(L2)
print L2
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print L2

L2 = tf.reshape(L2, [-1,7*7*64])

Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [5]:
W3 = tf.get_variable("W3",shape=[7*7*64,10], initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2,W3)+b

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

predicted = tf.equal(tf.argmax(hypothesis,1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(predicted,tf.float32))

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

training_epoch = 15
batch_size = 100

for epoch in range(training_epoch):
    total_batch = int(mnist.train.num_examples/batch_size)
    avg_cost = 0
    for step in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    print 'Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost)

print 'Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})


Epoch: 0001 cost = 0.455719731
Epoch: 0002 cost = 0.108034982
Epoch: 0003 cost = 0.073865837
Epoch: 0004 cost = 0.057227216
Epoch: 0005 cost = 0.047703753
Epoch: 0006 cost = 0.040097589
Epoch: 0007 cost = 0.035942509
Epoch: 0008 cost = 0.030597631
Epoch: 0009 cost = 0.026524342
Epoch: 0010 cost = 0.023042058
Epoch: 0011 cost = 0.020437390
Epoch: 0012 cost = 0.017877840
Epoch: 0013 cost = 0.016273823
Epoch: 0014 cost = 0.014223436
Epoch: 0015 cost = 0.012829466
Accuracy: